In [1]:
from pyjsg.validate_json import JSGPython

# JSG Syntax
The names of the various components defined in (Introducing JSON)[https://json.org/] are referenced in ***bold italics*** in the document below.  Example: A member definition defines the ***string***/***value*** pairs that may appear as ***members*** in an ***object*** definition.

A JSG Syntax definition consists of the following components:
1. Type and/or Ignore directives.  The `.TYPE` directive identifies the ***string** (if any) that identifies the JSG "type" of an ***object***.  It also names any JSG types that do not have a `.TYPE` identifier.  The `.IGNORE` directive identifies a set of ***strings*** that may appear in any JSON object.
2. Grammar rules.  These define valid (conformant) JSON ***objects*** and ***arrays***.
3. Lexer rules.  Lexer rules define the regular expressions that can be used to constrain the range of a JSON ***value***.


## Directives
### The `.TYPE` directive:

1. Names a unique property that identifies the JSG object being represented
2. (Optional) lists one or more production types that do not use the `.TYPE` discriminator

#### Syntax
`.TYPE <type> [ - <type> [<type>...]] ;`

#### No type directive

In [2]:
x = JSGPython('doc { a:., }')
print(x.conforms('{"a":"hello"}', 'T1'))
print(x.conforms('{"a":"test", "b": "work"}', 'T2'))
print(x.conforms('{"b":"test"}', 'T3'))

T1: Conforms to doc
T2: Conforms to doc
T3: FAIL - doc: Missing required field: 'a'


#### Object type identifier: " `.TYPE t  ;`"

In [3]:
x = JSGPython('''
.TYPE t ;
doc {a:.,}
id {b: @int}
''')
print(x.conforms('{"t": "doc", "a": {"t":"id", "b": 173}}', 'T1'))
print(x.conforms('{"t": "id", "b":"text"}', 'T2'))
print(x.conforms('{"t": "doc", "b": 173}', 'T3'))

T1: Conforms to doc
T2: FAIL - Invalid Integer value: "text"
T3: FAIL - doc: Missing required field: 'a'


#### Object type identifier with single exception: "`.TYPE t - id;`"

In [4]:
x = JSGPython('''
.TYPE t - id ;
doc {a:.,}
id {b: @int}
''')
print(x.conforms('{"t": "doc", "a": {"b": 173}}', 'T1'))
print(x.conforms('{"t": "id", "b": 140}', 'T2'))
print(x.conforms('{"b": "twelve"}', 'T3'))

T1: Conforms to doc
T2: Conforms to id
T3: FAIL - Invalid Integer value: "twelve"


#### Object type identifier with multiple exceptions: "`.TYPE t - id val;`"

In [5]:
x = JSGPython('''
.TYPE t - id val ;
doc {a:.,}
id {b: @int}
val {t: @number}
''')
print(x.conforms('{"t": "doc", "a": {"b": 173}}', 'T1'))
print(x.conforms('{"t": "id", "b": 140}', 'T2'))
print(x.conforms('{"t": 3.14}', 'T3'))

T1: Conforms to doc
T2: Conforms to id
T3: Conforms to val


### The `.IGNORE` directive
The `.IGNORE` directive identifies a list of property names to be globally ignored

#### Syntax
`.IGNORE <type> [<type> ...] ;`

#### Without Ignore

In [6]:
x = JSGPython('doc {a:@string}')
print(x.conforms('{"a":"hello"}'))
print(x.conforms('{"a":"hello", "target":"earthling"}'))

Conforms to doc
FAIL - Unknown attribute: target=earthling


In [7]:
x = JSGPython('''
.IGNORE target;
doc {a:@string}''')
print(x.conforms('{"a":"hello"}'))
print(x.conforms('{"a":"hello", "target":"earthling"}'))
print(x.conforms('{"a":"hello", "target":"earthling", "mode": "formal"}'))

Conforms to doc
Conforms to doc
FAIL - Unknown attribute: mode=formal


### Object Definitions
A JSG object definition consists of the definition name followed by the definition enclosed in curly braces ({ ... })

#### Syntax
`<identifier> : { [<element definitions>] }` 

Where identifier must either be a single, upper case character or, if there are two or more characters, at least one of them must be lower case alphabetic.

* valid identifiers:  A,  Type,  type, a17, a_abc, DOCx
* invalid identifiers: AA, A17, DOCX, A_

##### Simple object definition
An object definition with no element definitions describes an empty object

In [8]:
x = JSGPython('mtdoc {}')
print(x.conforms('{}'))
print(x.conforms('{"name":"Fred"}'))

# .IGNORE elements can always appear - even in empty documents
x = JSGPython('.IGNORE name; mtdoc {}')
print(x.conforms('{}'))
print(x.conforms('{"name":"Fred"}'))
print(x.conforms('{"location":"Spain"}'))

Conforms to mtdoc
FAIL - Unknown attribute: name=Fred
Conforms to mtdoc
Conforms to mtdoc
FAIL - Unknown attribute: location=Spain


### `element definitions` 
**`element definitions`** consist of either:

 1. a list of one or more `member definitions` or ...
 2. ... a `mapping definition`


#### `member definition`
A **`member definition`** identifies the string (name), type and possible values for a JSON object member.  `member definition` can also determine whether a member is required or optional.

#### Syntax
`<name> : <valueType> [<cardinality]` 
or
`<objectId>`
or
`(<name> <name> ...) : <valuetype> [<cardinality]`

** name / valuetype format **

In [18]:
x = JSGPython('''doc {
    last_name : @string,       # exactly one last name of type string
    first_name : @string+      # array or one or more first names
    age : @int?,               # optional age of type int
    weight : @number*          # array of zero or more weights
}
''')
print(x.conforms('''
{ "last_name" : "snooter",
  "first_name" : ["grunt", "peter"],
  "weight" : []
}'''))


FAIL - Invalid String value: "['grunt', 'peter']"
